In [33]:
pip install -U sagemaker

     |████████████████████████████████| 570 kB 23.0 MB/s eta 0:00:01
     |████████████████████████████████| 132 kB 64.7 MB/s eta 0:00:01
     |████████████████████████████████| 9.1 MB 48.0 MB/s eta 0:00:01
     |████████████████████████████████| 79 kB 851 kB/s  eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-2.108.0-py2.py3-none-any.whl size=786331 sha256=5f9aeef4f8c01a33f69490ef03c944061e82fc1edde8351e31b70939678a6f42
  Stored in directory: /root/.cache/pip/wheels/14/ff/39/57f8e3d429d0d851b64249b3bf1afd3a5e5bf41d564c4f3f5c
Successfully built sagemaker
  Attempting uninstall: botocore
    Found existing installation: botocore 1.20.89
    Uninstalling botocore-1.20.89:
      Successfully uninstalled botocore-1.20.89
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.4.2
    Uninstalling s3transfer-0.4.2:
      Successfully uninstalled s3transfer-0.4.2
  Attempting uninstall: boto3
    Found existing installation: boto3 1.17.89
    Uninstallin

In [34]:
import boto3
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import metrics
import joblib
import tarfile

import tensorflow as tf
from tensorflow import keras

import sagemaker
from sagemaker import get_execution_role

from sagemaker.sklearn.estimator import SKLearn
from sagemaker.sklearn.model import SKLearnModel
from sagemaker.tensorflow import TensorFlow, TensorFlowModel

In [35]:
sess = sagemaker.Session()
role = get_execution_role()

#### Deploy Original Model

In [4]:
export_dir = 'Banking_SVM_rbf_base.pkl'
with tarfile.open('Banking_SVM_rbf_base.tar.gz', mode='w:gz') as archive:
    archive.add(export_dir, recursive=True)

In [5]:
# upload the trained model to S3 bucket
prefix = "trained-model"
bucket = sess.default_bucket()
svc_model_loc = sess.upload_data(path="Banking_SVM_rbf_base.tar.gz", bucket=bucket, key_prefix=prefix)
print("Trained model artifact saved at:\n", svc_model_loc)

Trained model artifact saved at:
 s3://sagemaker-eu-central-1-901214976877/trained-model/Banking_SVM_rbf_base.tar.gz


In [6]:
from sagemaker.sklearn.model import SKLearnModel

svc_model = SKLearnModel(
    model_data=svc_model_loc,
    role=role,
    entry_point="aws_bank_orig.py",
    dependencies=['requirements.txt'],
    framework_version='0.23-1',
    py_version='py3'
)

In [7]:
svc_model.deploy(
    instance_type='ml.c4.xlarge',  # choose the right instance type
    initial_instance_count=1)

-------!

#### Deploy Defence Model - D1 (for Extraction)

In [27]:
export_dir = 'defense_model_banking_svc_scaled.pkl'
archive_name = 'defense_model_banking_svc_scaled.tar.gz'
with tarfile.open(archive_name, mode='w:gz') as archive:
    archive.add(export_dir, recursive=True)

In [28]:
# upload the trained model to S3 bucket
prefix = "trained-model"
bucket = sess.default_bucket()
svc_def1_model_loc = sess.upload_data(path="defense_model_banking_svc_scaled.tar.gz", bucket=bucket, key_prefix=prefix)
print("Defence model artifact saved at:\n", svc_def1_model_loc)

Defence model artifact saved at:
 s3://sagemaker-eu-central-1-901214976877/trained-model/defense_model_banking_svc_scaled.tar.gz


In [36]:
from sagemaker.sklearn.model import SKLearnModel

svc_def_model = SKLearnModel(
    model_data=svc_def1_model_loc,
    role=role,
    entry_point="aws_def_d1.py",
    framework_version='1.0-1',
    py_version='py3'
)

In [37]:
svc_def_model.deploy(
    instance_type='ml.c4.xlarge',  # choose the right instance type
    initial_instance_count=1)

INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2022-09-06-12-11-45-019
INFO:sagemaker:Creating endpoint with name sagemaker-scikit-learn-2022-09-06-12-11-45-537


------!

#### Deploy Defence Model - D2 (for Evasion)

In [17]:
loaded_defense_model = keras.models.load_model('Evasion_Def_D2.h5')

[2022-09-06 11:50:45.306 tensorflow-2-3-cpu-py-ml-t3-medium-f34868a61417544349a28b60e5e0:40 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-09-06 11:50:45.659 tensorflow-2-3-cpu-py-ml-t3-medium-f34868a61417544349a28b60e5e0:40 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.


In [18]:
model_version = 1
export_dir = 'Evasion_Def_D2/Servo/' + str(model_version)
tf.saved_model.save(loaded_defense_model, export_dir)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: Evasion_Def_D2/Servo/1/assets


INFO:tensorflow:Assets written to: Evasion_Def_D2/Servo/1/assets


In [19]:
with tarfile.open('Evasion_Def_D2.tar.gz', mode='w:gz') as archive:
    archive.add(export_dir, recursive=True)

In [20]:
# upload the trained model to S3 bucket
prefix = "trained-model"
bucket = sess.default_bucket()
evasion_def_d2_model_loc = sess.upload_data(path="Evasion_Def_D2.tar.gz", bucket=bucket, key_prefix=prefix)
print("Defence model artifact saved at:\n", evasion_def_d2_model_loc)

Defence model artifact saved at:
 s3://sagemaker-eu-central-1-901214976877/trained-model/Evasion_Def_D2.tar.gz


In [21]:
evasion_def_d2_mdl = TensorFlowModel(model_data=evasion_def_d2_model_loc, role=role, framework_version="2.3.1",)

In [22]:
evasion_def_d2_mdl.deploy(
    instance_type='ml.c4.xlarge',  # choose the right instance type
    initial_instance_count=1)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Creating model with name: tensorflow-inference-2022-09-06-11-50-52-426
INFO:sagemaker:Creating endpoint with name tensorflow-inference-2022-09-06-11-50-52-961


----!